In [0]:
import sys 
!{sys.executable} -m pip install tensorflow

In [0]:
import sys 
!{sys.executable} -m pip install opencv-python


In [0]:
import os
import numpy as np
import cv2
import keras.backend as K
import random
import tensorflow as tf


def load_image(Path = './valid/XR_FOREARM', size = 512):
    Images = []
    for path in Path:
        try:
            image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            Images.append(image)

        except Exception as e:
            print(str(e))

    Images = np.asarray(Images).astype('float32')
    Images=Images.reshape((len(Path),size,size,1))

    mean = np.mean(Images)			#normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    return Images

Using TensorFlow backend.


In [0]:
def load_path(root_path = './valid/XR_FOREARM', size = 512):
	'''
	load MURA dataset
	'''

	Path = []
	labels = []
	for root,dirs,files in os.walk(root_path): #Read all pictures, os.walk Return to iterator genertor Traverse all files
		for name in files:
			path_1 = os.path.join(root,name)
			Path.append(path_1)
			if root.split('_')[-1]=='positive':	 #positive Label is 1，otherwise 0；
				labels+=[1]   	          	 #Last level directory file patient11880\\study1_negative\\image3.png
			else:
			    labels+=[0]
	print (len(Path))
	labels = np.asarray(labels)
	return Path, labels


Path,labels=load_path(r'/content/MURA-v1.1/train/XR_FOREARM',64)
images=load_image(Path,64)

1825


In [0]:
    def train_model():
        model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu,input_shape=(64,64,1)),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),       
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(2,activation=tf.nn.softmax)
        ])
        model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
        model.fit(images,labels,epochs=15,verbose=2)
     
        return model

In [0]:
model=train_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/15
 - 2s - loss: 0.6805 - acc: 0.6285
Epoch 2/15
 - 2s - loss: 0.6401 - acc: 0.6422
Epoch 3/15
 - 2s - loss: 0.6347 - acc: 0.6663
Epoch 4/15
 - 2s - loss: 0.6274 - acc: 0.6784
Epoch 5/15
 - 2s - loss: 0.6078 - acc: 0.6904
Epoch 6/15
 - 2s - loss: 0.6019 - acc: 0.7058
Epoch 7/15
 - 2s - loss: 0.5689 - acc: 0.7205
Epoch 8/15
 - 2s - loss: 0.5599 - acc: 0.7277
Epoch 9/15
 - 2s - loss: 0.5269 - acc: 0.7441
Epoch 10/15
 - 2s - loss: 0.5094 - acc: 0.7518
Epoch 11/15
 - 2s - loss: 0.4681 - acc: 0.7874
Epoch 12/15
 - 2s - loss: 0.4381 - acc: 0.8027
Epoch 13/15
 - 2s - loss: 0.3964 - acc: 0.8290
Epoch 14/15
 - 2s - loss: 0.3653 - acc: 0.8367
Epoch 15/15
 - 2s - loss: 0.3295 - acc: 0.8510


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               401536    
__________

In [0]:
#Validation of test data
ValidPath,Validlabels=load_path(r'/content/MURA-v1.1/valid/XR_FOREARM',64)
Validimages=load_image(ValidPath,64)

301


In [0]:
score = model.evaluate(Validimages, Validlabels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8933030850071447
Test accuracy: 0.59468436


In [0]:
model.save('forearm.h5')